In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [0]:
session = SparkSession.builder.appName("StructuredNetworkWordCount").getOrCreate()

In [0]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

In [0]:
words = lines.select(
    explode(
        split(lines.value, " ")
    ).alias("word")
)

In [0]:
wordCounts = words.groupBy("word").count()

In [0]:
query = wordCounts.writeStream.outputMode("complete").format("console").start()

query.awaitTermination()

---------------------------------------------------------------------------
StreamingQueryException                   Traceback (most recent call last)
File <command-1357460101002405>:3
      1 query = wordCounts.writeStream.outputMode("complete").format("console").start()
----> 3 query.awaitTermination()

File /databricks/spark/python/pyspark/sql/streaming/query.py:198, in StreamingQuery.awaitTermination(self, timeout)
    196     return self._jsq.awaitTermination(int(timeout * 1000))
    197 else:
--> 198     return self._jsq.awaitTermination()

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *args)
   1315 command = proto.CALL_COMMAND_NAME +\
   1316     self.command_header +\
   1317     args_command +\
   1318     proto.END_COMMAND_PART
   1320 answer = self.gateway_client.send_command(command)
-> 1321 return_value = get_return_value(
   1322     answer, self.gateway_client, self.target_id, self.name)
   1324 for temp